In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import altair as alt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [5]:
# df = getDF('reviews_Electronics.json.gz')
#df = pd.read_csv('/content/drive/MyDrive/Professional/Break Through AI/BTAI Verizon Project/shortened_data.csv', index_col=[0])
df = pd.read_csv('drive/My Drive/amazon_reviews_data.csv', index_col=[0])

In [6]:
def sentiment(x):
    if x >= 4:
        return 1
    elif x >= 2:
        return 0
    else:
        return -1

In [7]:
df['sentiment'] = df['overall'].apply(sentiment)

In [8]:
df['reviewText'] = df['reviewText'].str.lower()

In [9]:
df.drop(['unixReviewTime', 'reviewTime'], axis=1, inplace=True)

In [10]:
from sklearn.feature_extraction.text import CountVectorizer

In [11]:
import nltk

In [12]:
from nltk.corpus import stopwords
nltk.download('stopwords')
stop = stopwords.words('english')

pat = r'\b(?:{})\b'.format('|'.join(stop))
df['processed_text'] = df['reviewText'].str.replace(pat, '')
df['processed_text'] = df['processed_text'].str.replace(r'\s+', ' ')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [13]:
vectorizer = CountVectorizer(min_df=10)

In [14]:
new_df = df.dropna()

In [15]:
X = vectorizer.fit_transform(new_df['processed_text'])

In [16]:
positive_reviews = new_df[new_df['sentiment'] == 1]
neutral_reviews = new_df[new_df['sentiment'] == 0]
negative_reviews = new_df[new_df['sentiment'] == -1]

In [17]:
pos_vectorizer = CountVectorizer(min_df=10)
neut_vectorizer = CountVectorizer(min_df=10)
neg_vectorizer = CountVectorizer(min_df=10)

In [18]:
pos_feats = pos_vectorizer.fit_transform(positive_reviews['processed_text'])
neut_feats = neut_vectorizer.fit_transform(neutral_reviews['processed_text'])
neg_feats = neg_vectorizer.fit_transform(negative_reviews['processed_text'])

In [ ]:
#https://www.nltk.org/book/ch05.html
#

# ADJ & ADV Feature

In [19]:
def descriptors(text):
  wants = ["VB", "JJ", "RB"]
  tokentext = nltk.word_tokenize(text)
  newlist = [t[0] for t in nltk.pos_tag(tokentext) if t[1] in wants]
  newtext = ' '.join(newlist)
  return newtext

In [20]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [21]:
new_df['adj&adv'] = new_df['reviewText'].apply(descriptors)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [22]:
new_df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,sentiment,processed_text,adj&adv
0,AKM1MP6P0OYPR,0132793040,"Vicki Gibson ""momo4""","[1, 1]",corey barker does a great job of explaining bl...,5.0,Very thorough,1,corey barker great job explaining blend modes ...,great great buy individually really just subsc...
1,A2CX7LUOHB2NDG,0321732944,Bernie,"[0, 0]",while many beginner dvds try to teach you ever...,5.0,Adobe Photoshop CS5 Crash Course with master P...,1,many beginner dvds try teach everything know ...,many teach know critical know start confident ...
2,A2NWSAGRHCP8N5,0439886341,bowmans2007,"[1, 1]",it never worked. my daughter worked to earn th...,1.0,absolutely horrible,-1,never worked. daughter worked earn money get ...,never earn get never buy new replace
3,A2WNBOD3WNDNKT,0439886341,JAL,"[1, 1]",some of the functions did not work properly. ...,3.0,Disappointing,0,functions work properly. daughter bought mone...,not work properly own not advertised
4,A1GI0U4ZRJA8WN,0439886341,Truthfull,"[4, 4]",do not waste your money on this thing it is te...,1.0,TERRIBLE DONT WASTE YOUR MONEY,-1,waste money thing terrible boutght product so...,do not waste terrible i remember always reset ...


In [23]:
vectorizer = CountVectorizer(min_df = 0.01, max_df=0.95)
X = vectorizer.fit_transform(new_df['adj&adv'])
y = new_df['sentiment']

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [25]:
model = MultinomialNB()
model.fit(X_train, y_train)

MultinomialNB()

In [26]:
preds = model.predict(X_test)

In [27]:
print(classification_report(y_test, preds))

              precision    recall  f1-score   support

          -1       0.54      0.50      0.52      5357
           0       0.59      0.65      0.62     10501
           1       0.69      0.63      0.66      8039

    accuracy                           0.61     23897
   macro avg       0.61      0.59      0.60     23897
weighted avg       0.61      0.61      0.61     23897



In [28]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test, preds))

[[2656 2297  404]
 [1752 6823 1926]
 [ 503 2462 5074]]


In [29]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(solver='lbfgs', alpha=1e-5,
hidden_layer_sizes=(15, 5), random_state=3)
clf.fit(X_train, y_train)
MLPClassifier(alpha=1e-05, hidden_layer_sizes=(15, 5), random_state=3,solver='lbfgs')
preds1 = clf.predict(X_test)
print(classification_report(y_test, preds1))

              precision    recall  f1-score   support

          -1       0.60      0.46      0.52      5357
           0       0.60      0.69      0.64     10501
           1       0.67      0.65      0.66      8039

    accuracy                           0.62     23897
   macro avg       0.62      0.60      0.61     23897
weighted avg       0.62      0.62      0.62     23897



/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


In [30]:
from sklearn.linear_model import SGDClassifier
clf = SGDClassifier(loss="hinge", penalty="l2", max_iter=50)
clf.fit(X_train, y_train)
SGDClassifier(max_iter=50)
clf = SGDClassifier(loss="log", max_iter=50).fit(X_train, y_train)
preds2 = clf.predict(X_test)
print(classification_report(y_test, preds2))

              precision    recall  f1-score   support

          -1       0.62      0.38      0.47      5357
           0       0.59      0.71      0.64     10501
           1       0.67      0.66      0.67      8039

    accuracy                           0.62     23897
   macro avg       0.63      0.58      0.59     23897
weighted avg       0.62      0.62      0.61     23897

